In [1]:
# !pip install aiohttp

In [2]:
import aiohttp
import asyncio
import requests
import numpy as np
import pandas as pd

import warnings as wn
wn.filterwarnings("ignore")

In [3]:
headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.2 Safari/605.1.15"
    }

In [4]:
# 1 - ROIC (10Y Median) -> Higher than 12% OR ROE (10Y Median) -> Higher than 15%
# 2 - P/E Ratio -> Lower than a Sector/Industry Median)
# 3 - EBIT/EV Ratio (1.0 / (EV/EBIT)) -> Higher than a risk-free rate (13 Weeks T-Bills))

In [5]:
def get_dataframe(data):
    cols, rows = data["headers"], data["rows"]
    df = pd.DataFrame(data=rows, columns=list(cols.keys()))
    df.columns = list(cols.values())
    df.set_index(df.columns[0], inplace=True)
    return df


def get_response(url):
    session = requests.Session()
    response = session.get(url=url, headers=headers)
    return response.json()


def get_tickers():
    url = f"https://api.nasdaq.com/api/screener/stocks?limit=0&marketcap=mega|large"
    session = requests.Session()
    response = session.get(url=url, headers=headers)
    data = response.json()
    df = get_dataframe(data["data"]["table"])
    if isinstance(df, pd.DataFrame):
        df.index = [i.replace("/", ".") for i in df.index]
    return df.index.tolist()


def get_risk_free_rate():
    url = "https://query1.finance.yahoo.com/v8/finance/chart/^IRX"
    response = get_response(url)
    return response["chart"]["result"][0]["meta"]["regularMarketPrice"] / 100


In [6]:
def get_profile(ticker, data):
    try:
        metadata = data["datasets"]["metadata"]
        if metadata["template_type"] == "normal":
            roic_10y_median = np.median([i[1] for i in data["datasets"]["chart"][-10:]])
            lbls = ["name", "exchange", "sector", "industry", "pe", "sector_pe_median", "industry_pe_median"]
            rows = [metadata[i] for i in lbls]
            rows[4:] = [float(i) for i in rows[4:]]
            lbls.extend(["roic_10y_median", "ebit_ev"])
            rows.extend([roic_10y_median, 1.0 / float(metadata["ev_ebit"])])
            df = pd.DataFrame(data=rows, index=lbls, columns=[ticker])
            return df
    except (KeyError, ValueError, ZeroDivisionError):
        pass


async def get_async_response(ticker, session):
    url = f"https://api.quickfs.net/stocks/{ticker}/ovr/Annual/?sortOrder=ASC"
    async with session.get(url, headers=headers) as response:
        data = await response.json()
        return get_profile(ticker, data)


async def main(tickers):
    async with aiohttp.ClientSession() as session:
        return pd.concat(
            await asyncio.gather(*[asyncio.create_task(get_async_response(ticker, session)) for ticker in tickers]),
            axis=1
        ).T

In [7]:
tickers = get_tickers()
df = await main(tickers)
df

,name,exchange,sector,industry,pe,sector_pe_median,industry_pe_median,roic_10y_median,ebit_ev
AAPL,Apple Inc,NASDAQ,Information Technology,"Technology Hardware, Storage & Peripherals",31.4,37.3,27.6,0.280653,0.037453
MSFT,Microsoft Corporation,NASDAQ,Information Technology,Software,35.9,37.3,37.3,0.220318,0.035088
GOOG,Alphabet Inc Class C,NASDAQ,Communication Services,Interactive Media & Services,25.6,30.0,30.0,0.161311,0.051546
GOOGL,Alphabet Inc Class A,NASDAQ,Communication Services,Interactive Media & Services,25.4,30.0,30.0,0.161311,0.052083
AMZN,Amazon.com Inc,NASDAQ,Consumer Discretionary,Internet & Direct Marketing Retail,307.7,34.0,58.6,0.068497,0.010081
...,...,...,...,...,...,...,...,...,...
USFD,US Foods Holding Corp,NYSE,Consumer Staples,Food & Staples Retailing,29.3,27.0,39.0,0.026179,0.051546
TPR,Tapestry Inc,NYSE,Consumer Discretionary,"Textiles, Apparel & Luxury Goods",11.4,34.0,34.0,0.134088,0.102041
SRPT,Sarepta Therapeutics Inc,NASDAQ,Health Care,Biotechnology,-9.0,29.0,27.1,-0.411871,-0.0625
KEP,Korea Electric Power Corp ADR,NYSE,Utilities,Electric Utilities,-0.6,23.2,23.8,0.009427,-3.333333


In [8]:
risk_free_rate = get_risk_free_rate()
df[(df['roic_10y_median'] > 0.12) & (df['ebit_ev'] > risk_free_rate) & (df['pe'] < df['industry_pe_median'])]

,name,exchange,sector,industry,pe,sector_pe_median,industry_pe_median,roic_10y_median,ebit_ev
TSM,Taiwan Semiconductor Manufacturing,NYSE,Information Technology,Semiconductors & Semiconductor Equipment,16.4,37.3,33.5,0.218247,0.072464
UNH,UnitedHealth Group Incorporated,NYSE,Health Care,Health Care Providers & Services,20.8,29.0,22.8,0.129454,0.060976
HD,Home Depot Inc,NYSE,Consumer Discretionary,Specialty Retail,18.8,34.0,24.3,0.315162,0.066667
MRK,Merck & Company Inc,NYSE,Health Care,Pharmaceuticals,21.2,29.0,23.9,0.120248,0.05618
BABA,Alibaba Group Holding Ltd,NYSE,Consumer Discretionary,Internet & Direct Marketing Retail,22.8,34.0,58.6,0.121304,0.08
...,...,...,...,...,...,...,...,...,...
CHRW,CH Robinson Worldwide Inc,NASDAQ,Industrials,Air Freight & Logistics,14.6,26.4,27.8,0.227093,0.089286
TTC,Toro Co,NYSE,Industrials,Machinery,21.4,26.4,24.8,0.251449,0.05618
UTHR,United Therapeutics Corporation,NASDAQ,Health Care,Biotechnology,14.8,29.0,27.1,0.168952,0.11236
ALLE,Allegion PLC,NYSE,Industrials,Building Products,21.3,26.4,28.0,0.151718,0.052356


In [9]:
df1 = df.copy()
df1 = df1[df1["pe"] >= 0]
df1["pe_sector_ratio"] = df1["pe"] / df1["sector_pe_median"]
df1["pe_industry_ratio"] = df1["pe"] / df1["industry_pe_median"]
df1["roic_score"] = df1["roic_10y_median"].rank(method="first", ascending=False)
df1["ebit_ev_score"] = df1["ebit_ev"].rank(method="first", ascending=False)
df1["pe_sector_score"] = df1["pe_sector_ratio"].rank(method="first")
df1["pe_industry_score"] = df1["pe_industry_ratio"].rank(method="first")
df1["total_score"] = 0.5 * df1["roic_score"] + 0.25 * df1["ebit_ev_score"] + 0.1 * df1["pe_sector_score"] + 0.15 * df1["pe_industry_score"]

In [10]:
df1["rank"] = df1["total_score"].rank(method="first").astype(int)
df1.sort_values(by = "rank")

,name,exchange,sector,industry,pe,sector_pe_median,industry_pe_median,roic_10y_median,ebit_ev,pe_sector_ratio,pe_industry_ratio,roic_score,ebit_ev_score,pe_sector_score,pe_industry_score,total_score,rank
HPQ,HP Inc,NYSE,Information Technology,"Technology Hardware, Storage & Peripherals",13.1,37.3,27.6,0.525207,0.102041,0.351206,0.474638,5.0,98.0,75.0,124.0,53.10,1
BTI,British American Tobacco PLC ADR,NYSE,Consumer Staples,Tobacco,9.2,27.0,17.3,0.273541,0.15625,0.340741,0.531792,31.0,47.0,69.0,153.0,57.10,2
LYB,LyondellBasell Industries NV,NYSE,Materials,Chemicals,9.8,18.7,27.8,0.236904,0.116279,0.524064,0.352518,52.0,76.0,151.0,71.0,70.75,3
QCOM,Qualcomm Incorporated,NASDAQ,Information Technology,Semiconductors & Semiconductor Equipment,12.5,37.3,33.5,0.204263,0.09009,0.335121,0.373134,73.0,117.0,65.0,81.0,84.40,4
EXPD,Expeditors International of Washington Inc,NASDAQ,Industrials,Air Freight & Logistics,15.2,26.4,27.8,0.255208,0.1,0.575758,0.546763,36.0,100.0,183.0,161.0,85.45,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BMRN,Biomarin Pharmaceutical Inc,NASDAQ,Health Care,Biotechnology,223.4,29.0,27.1,-0.025669,0.004369,7.703448,8.243542,583.0,576.0,577.0,580.0,580.20,588
NOW,ServiceNow Inc,NYSE,Information Technology,Software,286.0,37.3,37.3,-0.048211,0.003713,7.66756,7.66756,585.0,579.0,576.0,579.0,581.70,589
PANW,Palo Alto Networks Inc,NASDAQ,Information Technology,Software,351.6,37.3,37.3,-0.114847,0.00208,9.426273,9.426273,588.0,582.0,580.0,581.0,584.65,590
PODD,Insulet Corporation,NASDAQ,Health Care,Health Care Equipment & Supplies,33167.1,29.0,44.3,-0.022675,0.001327,1143.693103,748.693002,582.0,585.0,592.0,592.0,585.25,591


In [11]:
df1.sort_values("rank").to_csv("magic_formula.csv", encoding="utf-8-sig")